**Respiradores por município, dados do DataSUS**

Libraries

In [0]:
import pandas as pd
import numpy as np

###Leitura de CSV's

Leitos por município

In [2]:
respiradores = pd.read_csv("https://raw.githubusercontent.com/fraterno20/fraterno20-python-notebooks/master/Dataset/Dados-Primarios/respiradores_municipio.csv", sep = ";", encoding = "iso-8859-1")
respiradores.columns = ["city","quantidade_existente","quantidade_em_uso","estab_c_eq_sus"]
respiradores.head(2)

,city,quantidade_existente,quantidade_em_uso,estab_c_eq_sus
0,110001 Alta Floresta D'Oeste,-,-,-
1,110037 Alto Alegre dos Parecis,-,-,-


Dados de Lagitude e Longitude dos municípios 

In [3]:
city_lat_long =  pd.read_csv("https://raw.githubusercontent.com/fraterno20/fraterno20-python-notebooks/master/Dataset/Dados-Primarios/lat_long_municipios.csv")
city_lat_long.drop(columns = ["capital","Unnamed: 0"], inplace = True)
city_lat_long.rename(columns = {"nome":"city"},inplace = True)
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf
0,5200050,Abadia de Goiás,-16.7573,-49.4412,52
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,31


Base do IBGE, contendo código do município, nome do estado, do município, e o código do estado

In [4]:
data_ibge = pd.read_csv("https://raw.githubusercontent.com/fraterno20/fraterno20-python-notebooks/master/Dataset/Dados-Primarios/dados_ibge.csv")
data_ibge.drop(columns = ["Mesorregião Geográfica","Nome_Mesorregião","Microrregião Geográfica","Nome_Microrregião","Município","Unnamed: 0"], inplace = True)
data_ibge.rename(columns = {"Nome_Município":"city","Nome_UF":"state","UF":"codigo_uf","Código Município Completo":"codigo_ibge"},inplace = True)
data_ibge.head(2)

,codigo_uf,state,codigo_ibge,city
0,11,Rondônia,1100015,Alta Floresta D'Oeste
1,11,Rondônia,1100379,Alto Alegre dos Parecis


###Manipulação de dataframes

Juntando a tabela da localização geográfica com a dos dados do ibge

In [5]:
city_lat_long = city_lat_long.merge(data_ibge, on = ["city","codigo_ibge","codigo_uf"],sort = "city")
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,5200050,Abadia de Goiás,-16.7573,-49.4412,52,Goiás
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,31,Minas Gerais


Usando Split para retirar o código do município

In [6]:
new = respiradores["city"].str.split(" ", n = 1, expand = True)
respiradores["codigo_ibge"] = new[0] #Dev sumiu com o ultimo digito de todos os codigos
respiradores.drop(columns=["city"], inplace = True)
respiradores.head(2)

,quantidade_existente,quantidade_em_uso,estab_c_eq_sus,codigo_ibge
0,-,-,-,110001
1,-,-,-,110037


Retirando o último digito para manter a igualdade, já que os dados da datasus vieram faltando o útltimo dígito

In [7]:
city_lat_long["codigo_ibge"] = city_lat_long["codigo_ibge"].map(lambda x: str(x)[:-1])
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,520005,Abadia de Goiás,-16.7573,-49.4412,52,Goiás
1,310010,Abadia dos Dourados,-18.4831,-47.3916,31,Minas Gerais


Merge com a localização 

In [8]:
respiradores = respiradores.merge(city_lat_long, on = ["codigo_ibge"], sort = "city", how = "outer")
respiradores.dropna(subset=["state"], inplace = True)
respiradores.head(2)

,quantidade_existente,quantidade_em_uso,estab_c_eq_sus,codigo_ibge,city,latitude,longitude,codigo_uf,state
2,-,-,-,110001,Alta Floresta D'Oeste,-11.92830,-61.9953,11.0,Rondônia
3,49,49,7,110002,Ariquemes,-9.90571,-63.0325,11.0,Rondônia


Retirando valores nulos

In [9]:
respiradores.replace("-", 0, inplace = True)
respiradores.replace(np.nan, 0, inplace = True)
respiradores.head(2)

,quantidade_existente,quantidade_em_uso,estab_c_eq_sus,codigo_ibge,city,latitude,longitude,codigo_uf,state
2,0,0,0,110001,Alta Floresta D'Oeste,-11.92830,-61.9953,11.0,Rondônia
3,49,49,7,110002,Ariquemes,-9.90571,-63.0325,11.0,Rondônia


###Download CSV

In [0]:
respiradores.to_csv("respiradores_municipio.csv")